In [1]:
import cv2
import numpy as np
from tqdm import tqdm
from glob import glob
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
ResNet50_model = ResNet50(weights='imagenet')
ResNet50_model_base = ResNet50(weights='imagenet', include_top=False)
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt2.xml')
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

In [3]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

def extract_Resnet50(tensor):
    return ResNet50_model_base.predict(preprocess_input(tensor))

model_classifier = Sequential()
model_classifier.add(GlobalAveragePooling2D(input_shape = ResNet50_model_base.output_shape[1:]))
model_classifier.add(Dense(133, activation = 'softmax'))
model_classifier.load_weights('saved_models/weights.best.Resnet50.hdf5')

def Resnet50_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = model_classifier.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

def classify_dog_or_human(img_path):
    if(dog_detector(img_path)):
        print("Dog detected! Predicting breed...")
        return Resnet50_predict_breed(img_path)
    elif(face_detector2(img_path)):
        print("Human detected! Predicting possible breed...")
        return Resnet50_predict_breed(img_path)
    else:
        print("This image does not contain any human or dog!")

In [4]:
import matplotlib.pyplot as plt
def show_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)

In [5]:
img_path = 'test_images/dog_senbernar.jpg'
show_image(img_path)

In [6]:
classify_dog_or_human(img_path)

Dog detected! Predicting breed...


'Saint_bernard'